# **Forecasting IY's Inventory Purchases**

A model by: <br>
Cristina Esposito and Alya Gabsi
____________________________________________

In [1]:
# Importing Dependencies

!pip install pmdarima # for arima
!pip install --upgrade -q gspread

# need to import to authenticate and approve the use of your google account
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

# required for data manipulation
import pandas as pd
import numpy as np 

# Optimization Packages
!pip install pulp
from pulp import * 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 15.0 MB/s 
     |████████████████████████████████| 9.8 MB 60.7 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 15.3 MB/s 


## **Part 1: Data Preparation**

### 1. Importing the datasets to be used within the forecasting model

In [2]:
# authorize your google account and permission to link with the google sheet
gc = gspread.authorize(creds)

# importing the master google sheet
data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YMT179qBif4-up6Xc0TthVWXzMJbJ5n2PQ0olPOCq9I/edit?usp=sharing') 

In [3]:
# selecting the "Products sold" in the master google sheet
worksheet = data.worksheet('Products Sold') 

# get_all_values gives a list of rows
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
df=pd.DataFrame.from_records(rows)

new_header = df.iloc[3] #grab the fourth row to use as the header of the dataframe
df = df[4:] #take the data from the fifth row onwards
df.columns = new_header #set the header row as the dataframe header

df.head(10) #showing a sample of the "Products sold" dataframe tab that was imported

3,Product,To forecast?,2021-07-09,2021-07-23,2021-08-13,2021-08-27,2021-09-24,2021-10-22,2021-11-26,2021-12-17,2022-01-14,2022-02-11,2022-03-25,2022-04-22,2022-05-13,2022-06-06,2022-06-20,2022-07-04,2022-07-18
4,Apples,Yes,586,187,279,58,0,344,523,637,690,525,667,470,840,209,192,267,193
5,Asparagus,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,57,0
6,Bananas,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Beans,Yes,94,19,183,53,0,112,188,0,0,7,8,0,0,0,0,0,0
8,Beets,Yes,185,100,261,96,0,139,0,321,0,91,68,0,117,0,0,0,0
9,Broccoli,Yes,3,34,25,54,46,0,0,0,0,27,40,0,77.22,0,0,0,0
10,Brussels Sprouts,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,Cabbage,Yes,20,56,32,15,0,46,0,59,0,16,0,0,0,0,0,0,0
12,Canteloupe,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,Carrots,Yes,364,799,606,1037,361,607,440,1113,889,1035,1336,1141,1168,171,139,143,140


In [4]:
# importing the external data to be used within some forecasting models, taken from the "External Data" tab in the master google sheet
worksheet = data.worksheet('External Data') 

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
ex=pd.DataFrame.from_records(rows)

new_header = ex.iloc[0] #grab the first row for the header
ex = ex[1:] #take the data from the second row onward
ex.columns = new_header #set the header row as the df header

ex.head(10)

,Date of the Market,Temperature,Precipitation,Apples,Asparagus,Bananas,Beans,Beets,Bell Peppers,Blueberries,...,Sprouts,Squash,Strawberries,Sugar,Sweet Potatoes,Swiss Chard,Tomatoes,Turnip,Watermelon,Zucchini
1,2021-07-09,18.5,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,0,1
2,2021-07-23,21.4,0,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,0,1
3,2021-08-13,26.7,0,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,2021-08-27,21.3,0,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,2021-09-24,18.4,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,2021-10-22,8.9,1,1,0,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,0
7,2021-11-26,0.7,1,1,0,1,0,1,0,0,...,1,1,0,1,1,0,1,1,0,0
8,2021-12-17,4.7,0,1,0,1,0,1,0,0,...,1,1,0,1,1,0,0,1,0,0
9,2022-01-14,-14.8,0,1,0,1,0,1,0,0,...,1,1,0,1,1,0,0,1,0,0
10,2022-02-11,2.6,1,1,0,1,0,1,0,0,...,1,1,0,1,1,0,0,1,0,0


In [5]:
# importing the conversion guide to convert the sold units to purchased units 
worksheet = data.worksheet('Conversion') 

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
conv=pd.DataFrame.from_records(rows)

new_header = conv.iloc[0] #grab the first row for the header
conv = conv[1:] #take the second row onward for the data
conv.columns = new_header #set the header row as the df header

# selecting only the important columns - product name and the conversion value
conv=conv[["Product","Conversion"]]
conv.head(10)

,Product,Conversion
1,Apples,0.3333333333
2,Beans,0.18
3,Beets,0.3333333333
4,Broccoli,1
5,Cabbage,2
6,Canteloupe,1
7,Carrots,0.2
8,Cauliflower,1
9,Celery,1
10,Chickpeas,0.211


### 2. Data cleaning/formatting

In [6]:
# selecting only the products you want to forecast, where "To Forecast?" is "Yes" and storing it within another dataframe
df2=df[df["To forecast?"]=="Yes"] 
df2.head(10)

3,Product,To forecast?,2021-07-09,2021-07-23,2021-08-13,2021-08-27,2021-09-24,2021-10-22,2021-11-26,2021-12-17,2022-01-14,2022-02-11,2022-03-25,2022-04-22,2022-05-13,2022-06-06,2022-06-20,2022-07-04,2022-07-18
4,Apples,Yes,586,187,279,58,0,344,523,637,690,525,667,470,840,209,192,267,193
7,Beans,Yes,94,19,183,53,0,112,188,0,0,7,8,0,0,0,0,0,0
8,Beets,Yes,185,100,261,96,0,139,0,321,0,91,68,0,117,0,0,0,0
9,Broccoli,Yes,3,34,25,54,46,0,0,0,0,27,40,0,77.22,0,0,0,0
11,Cabbage,Yes,20,56,32,15,0,46,0,59,0,16,0,0,0,0,0,0,0
13,Carrots,Yes,364,799,606,1037,361,607,440,1113,889,1035,1336,1141,1168,171,139,143,140
15,Celery,Yes,23,57,20,26,26,29,16,65,35,36,54,35,14,37,0,0,0
16,Chickpeas,Yes,58,33,3,56,0,58,43,54,43,42,54,66,53,0,0,0,0
19,Cucumber,Yes,8,70,60,80,48,0,0,0,0,0,0,0,36,75,86,93,111.5
21,Eggs,Yes,230,205,178,144,123,117,199,182,298,232,166,157,120,56,75,83,0


In [7]:
# since we filtered to have only the products we want to forecast, we can drop the "To forecast?" column and storing it in another dataframe
df3=df2.drop(columns=["To forecast?"]) 
df3.head(10)

3,Product,2021-07-09,2021-07-23,2021-08-13,2021-08-27,2021-09-24,2021-10-22,2021-11-26,2021-12-17,2022-01-14,2022-02-11,2022-03-25,2022-04-22,2022-05-13,2022-06-06,2022-06-20,2022-07-04,2022-07-18
4,Apples,586,187,279,58,0,344,523,637,690,525,667,470,840,209,192,267,193
7,Beans,94,19,183,53,0,112,188,0,0,7,8,0,0,0,0,0,0
8,Beets,185,100,261,96,0,139,0,321,0,91,68,0,117,0,0,0,0
9,Broccoli,3,34,25,54,46,0,0,0,0,27,40,0,77.22,0,0,0,0
11,Cabbage,20,56,32,15,0,46,0,59,0,16,0,0,0,0,0,0,0
13,Carrots,364,799,606,1037,361,607,440,1113,889,1035,1336,1141,1168,171,139,143,140
15,Celery,23,57,20,26,26,29,16,65,35,36,54,35,14,37,0,0,0
16,Chickpeas,58,33,3,56,0,58,43,54,43,42,54,66,53,0,0,0,0
19,Cucumber,8,70,60,80,48,0,0,0,0,0,0,0,36,75,86,93,111.5
21,Eggs,230,205,178,144,123,117,199,182,298,232,166,157,120,56,75,83,0


In [8]:
# changing the index value of the external data to be the date of the market
ex=ex.set_index("Date of the Market")

In [9]:
#when importing the external data tab, it does not maintain the appropriate data types for the columns. Here we convert the temperature as a float value and al other columns as integer values
ex.Temperature=ex.Temperature.astype(float) 
ex.iloc[:,1:]=ex.iloc[:,1:].astype(int) 

In [10]:
# splitting the external data to have the values needed for training vs. prediction
extrain=ex.iloc[:-1,:]
expred=ex.iloc[-1:,:]

In [11]:
expred

,Temperature,Precipitation,Apples,Asparagus,Bananas,Beans,Beets,Bell Peppers,Blueberries,Broccoli,...,Sprouts,Squash,Strawberries,Sugar,Sweet Potatoes,Swiss Chard,Tomatoes,Turnip,Watermelon,Zucchini
Date of the Market,,,,,,,,,,,,,,,,,,,,,
2022-08-01,30.0,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [12]:
# since we require to have information on the forecasted farmer's market entered within the external data tab, we grab that information to be used within the prediction
weather=expred.iloc[:,0]
precip=expred.iloc[:,1]

## **Part 2: Demand forecast models**

In [13]:
numproducts=len(df3) # from the dataframe that has only the products we want to forecast, here we store the number of products we are forecasting to be used within a for loop

In [14]:
# importing all the libraries to be used for our forecasting models
import statsmodels.api as sm # for moving average
from statsmodels.tsa.arima_model import ARIMA 
import pmdarima as pm # for auto arima
from statsmodels.tsa.holtwinters import SimpleExpSmoothing # for simple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing # for holts winters exponential smoothing
from sklearn.ensemble import RandomForestRegressor # for the random forest
import numpy as np # for the time variable in the regression
from sklearn.linear_model import LinearRegression # for linear regression

from sklearn.metrics import mean_absolute_percentage_error # for model assessment

In [15]:
df3

3,Product,2021-07-09,2021-07-23,2021-08-13,2021-08-27,2021-09-24,2021-10-22,2021-11-26,2021-12-17,2022-01-14,2022-02-11,2022-03-25,2022-04-22,2022-05-13,2022-06-06,2022-06-20,2022-07-04,2022-07-18
4,Apples,586,187,279,58,0,344,523,637,690,525,667,470,840,209,192,267,193
7,Beans,94,19,183,53,0,112,188,0,0,7,8,0,0,0,0,0,0
8,Beets,185,100,261,96,0,139,0,321,0,91,68,0,117,0,0,0,0
9,Broccoli,3,34,25,54,46,0,0,0,0,27,40,0,77.22,0,0,0,0
11,Cabbage,20,56,32,15,0,46,0,59,0,16,0,0,0,0,0,0,0
13,Carrots,364,799,606,1037,361,607,440,1113,889,1035,1336,1141,1168,171,139,143,140
15,Celery,23,57,20,26,26,29,16,65,35,36,54,35,14,37,0,0,0
16,Chickpeas,58,33,3,56,0,58,43,54,43,42,54,66,53,0,0,0,0
19,Cucumber,8,70,60,80,48,0,0,0,0,0,0,0,36,75,86,93,111.5
21,Eggs,230,205,178,144,123,117,199,182,298,232,166,157,120,56,75,83,0


In [17]:
products={} # define empty dictionary to store predictions

# here we have a for loop used for a model competition to produce the most accurate forecast for each product - accuracy is based on MAPE as it is an 
# easy to comprehend measure of accuracy for the end users

# for each product we go through this loop to generate the forecast
for i in range(numproducts):

  # restructuring the dataframe to have univariate time series format
  df4=pd.DataFrame(df3.iloc[i,:])
  product=df4.iloc[0,0] # get the product name

  # get the conversion factor of the product
  ind=int(conv[conv["Product"]==product].index.values)
  conversion=float(conv.iloc[ind-1,1]) # ensuring that the value is in float format

  df4["Date"]=df4.index # renaming the index as the date
  df4=df4[1:]
  df4.columns=["Sales","Date"] # renaming the columns to be date and sales
  df4 = df4.reset_index(drop=True) # dropping the index values to ensure the date is just the index
  df4=df4.sort_values(by="Date") # putting the dates from oldest to newest order
  df4=df4.set_index("Date")
  df4["Sales"]=df4["Sales"].astype(float) # have to convert the sales into a float type


  # splitting the dataset into train and test for the model competition
  train_num=int(round(len(df4)-1,0)) # we want to use everything up until the last row in the dataframe
  train=df4.iloc[:train_num,:]
  test=df4.iloc[train_num:,:]


  #-----------------------------------------------------------
  #             FORECASTING MODELS
  #-----------------------------------------------------------

  # Model 0 = moving average
  mode_ma=sm.tsa.ARIMA(train,order=(0,0,1)) # setting up the moving average model
  model_fit_ma = mode_ma.fit() # fitting the model

  ma = pd.DataFrame(model_fit_ma.predict(start=len(train)+1, end=len(df4))) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  ma.columns=["MA"] # naming the column as MA for moving average results
  ma=ma.reset_index(drop=True) # resetting the index so that it can be merged with the other model results


  # Model 1 = auto arima
  model_aa=pm.auto_arima(df4, start_p=1, start_q=1, 
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

  aa = pd.DataFrame(model_aa.predict(len(test))) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  aa.columns=["AA"] # naming the column as AA for auto aarima results
  aa=aa.reset_index(drop=True) # resetting the index so that it can be merged with the other model results


  # Model 2 = seasonal auto arima
  model_sa=pm.auto_arima(df4, start_p=1, start_q=1, 
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

  sa = pd.DataFrame(model_sa.predict(len(test))) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  sa.columns=["SA"] # naming the column as SA for seasonal arima results
  sa=sa.reset_index(drop=True) # resetting the index so that it can be merged with the other model results



  # Model 3 = simple exponential smoothing
  model_se=SimpleExpSmoothing(df4) # setting up the seasonal exponential model
  model_fit_se=model_se.fit() # fitting the model
  se = pd.DataFrame(model_fit_se.predict(start=len(train)+1, end=len(df4))) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  se.columns=["SE"] # naming the column as SE for simple exponential smoothing results
  se=se.reset_index(drop=True) # resetting the index so that it can be merged with the other model results



  # Model 4 = holt winter's exponential smoothing
  model_he=ExponentialSmoothing(df4) # setting up the holt's winter's exponential smoothing model
  model_fit_he=model_he.fit() # fitting the model
  he = pd.DataFrame(model_fit_he.predict(start=len(train)+1, end=len(df4))) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  he.columns=["HE"] # naming the column as HE for holt's exponential results
  he=he.reset_index(drop=True) # resetting the index so that it can be merged with the other model results



  # Model 5 = naive forecasting
  na = pd.DataFrame(train.Sales[len(train)-1].repeat(len(test))) #  taking the previous period's actuals as the forecast and storing it in a dataframe
  na.columns=["NA"] # naming the column as HE for holt's exponential results
  na=na.reset_index(drop=True) # resetting the index so that it can be merged with the other model results



  #------------------ SUPERVISED MODELS----------------------------

  inseason=int(expred[product]) # getting the inseason value for the period we are predicting for the product and ensuring that the in season external data value is set as integer

  df5=df4 # copying the dataframe in another dataframe
  temp=extrain[["Temperature", "Precipitation"]] # selecting the temperature and precipitation columns from the external data
  df6=pd.concat([df5,temp,extrain[product]], axis = 1) # adding the temperature, precipitation, and the specific product's in-season value columns to the sales dataframe

  df6["Sales-1"]=df6.Sales.shift(1) # shifting the sales values by 1 period
  df6["Sales-2"]=df6.Sales.shift(2) # shifting the sales values by 2 periods
  df6["Sales-3"]=df6.Sales.shift(3) # shifting the sales values by 3 periods
  df6=df6.dropna() # have to drop na values from the shift or else the model won't run on na values

  # splitting the dataset into train and test for the model competition
  train_num=int(round(len(df6)-1,0)) 
  train=df6.iloc[:train_num,:]
  X_train=train.iloc[:,1:] # selecting your X values from the train dataset
  y_train=train["Sales"] # selecting your sales target value  from the train dataset

  test=df6.iloc[train_num:,:]
  X_test=test.iloc[:,1:] # selecting your X values from the test dataset
  y_test=test["Sales"] # selecting your sales target value  from the test dataset


  # Model 6 = Random Forest
  rf = RandomForestRegressor(n_estimators=100, random_state=1) # setting up the random forest model
  rf_fit=rf.fit(X_train,y_train) # fitting the model

  rf = pd.DataFrame(rf_fit.predict(X_test)) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  rf.columns=["RF"] # naming the column as RF for random forest results
  rf=rf.reset_index(drop=True) # resetting the index so that it can be merged with the other model results


  # Model 7 = Linear regression
  df7=df6 # copying the dataframe in another dataframe
  df7["time"]=np.arange(len(df7.index)) # creating a new column called "time" for the seasonal component of a demand model in a linear regression

  train_num=int(round(len(df7)-1,0)) # splitting the data
  train=df7.iloc[:train_num,:] 
  X_train=train.iloc[:,1:] # selecting your X values from the train dataset
  y_train=train["Sales"] # selecting your sales target value  from the train dataset

  test=df7.iloc[train_num:,:]
  X_test=test.iloc[:,1:] # selecting your X values from the test dataset
  y_test=test["Sales"] # selecting your sales target value  from the test dataset


  lr = LinearRegression() # setting up the linear regression model
  lr_fit=lr.fit(X_train, y_train) # fitting the model

  lr = pd.DataFrame(lr_fit.predict(X_test)) # using the model to predict the periods based on the test dataset and storing it in a dataframe
  lr.columns=["LR"] # naming the column as LR for linear regression results
  lr=lr.reset_index(drop=True) # resetting the index so that it can be merged with the other model results



  # Assesing the best model using MAPE
  predictions=pd.concat([test.reset_index(), ma,aa,sa,se,he,na,rf,lr], axis=1)
  

  mape=[mean_absolute_percentage_error(predictions["Sales"],predictions["MA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["AA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["SA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["SE"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["HE"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["NA"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["RF"]),
        mean_absolute_percentage_error(predictions["Sales"],predictions["LR"])]
  

  best_model=mape.index(min(mape)) # selecting the index of the best model


  #------------------------------------------------------------

  # Predicting the value for the next market based on the most accurate model

  # If the best model is the moving average
  if best_model==0:
    forecast = int(model_fit_ma.predict(len(df4)))

  # If the best model is the auto arima
  elif best_model==1:
    forecast = int(model_aa.predict(1))

  # If the best model is the seasonal auto arima
  elif best_model==2:
    forecast = int(model_sa.predict(1))
  
  # If the best model is the simple exponential smoothing
  elif best_model==3:
    forecast = int(model_fit_se.predict(len(df4)))

  # If the best model is the Holt's winter's exponential smoothing
  elif best_model==4:
    forecast = int(model_fit_he.predict(len(df4)))

  # If the best model is naive
  elif best_model==5:
    forecast=int(df4.Sales[len(df4)-1])

  # If the best model is the random forest
  elif best_model==6:
    forecast = int(rf_fit.predict([[weather,precip,inseason,df6["Sales"][-1],df6["Sales"][-2],df6["Sales"][-3]]]))

  # If the best model if the regression
  else:
    forecast = int(lr_fit.predict([[weather,precip,inseason,df7["Sales"][-1],df7["Sales"][-2],df7["Sales"][-3],len(df7)]]))


  # converting the forecast value
  forecast=round(forecast*conversion)

  # storing the best forecasted value in the dictionary - to be used within the optimization model
  products[product]= 0 if forecast<0 else forecast # if there's a model that produces a negative value, then we convert it to 0 


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-pac

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=227.181, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=219.352, Time=0.16 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.05 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=225.223, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=219.368, Time=0.09 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.57 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=217.429, Time=0.10 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=217.497, Time=0.33 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.26 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.08 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.31 sec

Best model:  ARIMA(1,2,0)(0,0,0)[0]          
Total fit time: 2.303 seconds
Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=inf, Time=0.26 sec
 ARIMA(0,2,0)(0,0,0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=190.101, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=189.820, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=188.185, Time=0.01 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.410 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,1)[12] intercept   : AIC=inf, Time=0.52 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=190.101, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=191.818, Time=0.21 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=inf, Time=0.45 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=188.185, Time=0.03 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=192.100, Time=0.14 sec
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=192.100, Time=0.10 sec
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=194.100, Time=0.07 sec


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

...

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=171.834, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=167.393, Time=0.03 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.04 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=169.834, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=161.953, Time=0.06 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=163.392, Time=0.06 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=159.981, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=165.393, Time=0.02 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=161.455, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.06 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec

Best mode

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/ts

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=130.635, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=125.784, Time=0.04 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=128.637, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=127.727, Time=0.04 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=123.871, Time=0.01 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=125.807, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.04 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.08 sec

Best model:  ARIMA(1,2,0)(0,0,0)[0]          
Total fit time: 0.567 seconds
Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=inf, Time=0.16 sec
 ARIMA(0,2,0)(0,0,0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


In [18]:
products # forecasted values for the optimizer, these are in the units purchased (i.e. if apples are purchased in pounds, the value is in pounds)

{'Apples': 72,
 'Beans': 0,
 'Beets': 0,
 'Broccoli': 0,
 'Cabbage': 0,
 'Carrots': 28,
 'Celery': 0,
 'Chickpeas': 0,
 'Cucumber': 111,
 'Eggs': 5,
 'Garlic': 11,
 'Ginger': 13,
 'Lentils': 0,
 'Milk': 0,
 'Onions': 74,
 'Oranges': 104,
 'Potatoes': 80,
 'Rice': 0,
 'Salad': 1,
 'Squash': 0}


## **Part 3: Optimizer**


### Get Data 

In [19]:
# From notebook 

M = 10e9 # Defining big M for model
D = products # to get forecasted demands 
product_list = [product for product in D.keys()] # to get product list 

In [20]:
# From Google Sheets 

## To get Garden (G) and previous Left-overs (L) information

# Open the specified worksheet
client = gspread.authorize(creds)
sheet = client.open("MAIN FARMER'S MARKET").worksheet('Inventory')
# get_all_values gives a list of rows.
rows = sheet.get_all_values()
# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

# Defining function 

def f(x):
    dic = {}
    x = x.sort_values(by = 'Date of the Market', ascending = False)
    dic['Garden'] = x['Garden'].iloc[0]
    dic['Leftovers'] = x['Inventory Prior For Resale'].iloc[0]
    return pd.Series(dic, index=['Garden', 'Leftovers'])

free_inventory = df.groupby('Product').apply(f).reset_index()
free_inventory = free_inventory[free_inventory['Product'].isin(product_list)]
free_inventory = free_inventory.merge(conv, on='Product', how = 'left')

free_inventory['Garden'] = free_inventory['Garden'].apply(float)
free_inventory['Conversion'] = free_inventory['Conversion'].apply(float)
free_inventory['Leftovers'] = free_inventory['Leftovers'].apply(float)

free_inventory['Garden'] = free_inventory['Garden'] * free_inventory['Conversion']
free_inventory['Leftovers'] = free_inventory['Leftovers'] * free_inventory['Conversion']


free_inventory.set_index('Product', inplace=True)
# Get Garden data 
G = free_inventory['Garden'].to_dict()

# # Get leftovers from previous market 
L = free_inventory['Leftovers'].to_dict()


## To get Suppliers information (Inventory I and Quotes q) and budget (B) 

sheet = client.open("MAIN FARMER'S MARKET").worksheet('Supplier Data')
# get_all_values gives a list of rows.
rows = sheet.get_all_values()
# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

B = float(df.iloc[1,1])

df = df.rename(columns=df.iloc[2]).drop(df.index[0:3])
df.set_index(['Product Name'], inplace=True)
df['Supplier Inventory'] = df['Supplier Inventory'].apply(float)
df['Quote (in respective units)'] = df['Quote (in respective units)'].apply(float)

I = df['Supplier Inventory'].to_dict() # Getting supplier inventory
q = df['Quote (in respective units)'].to_dict() # Getting supplier quote


### Formulating and Solving Problem

Initializating the model

In [21]:
model = LpProblem('Order_Optimization', LpMinimize)

Setting Decision Variables 

In [22]:
# Setting Decision Variables 

## Amount to order from supplier 
X = LpVariable.dicts('Supplied', product_list, lowBound = 0, cat = 'Continuous')

##  Sales 
S = LpVariable.dicts('Sales', product_list, lowBound =0, cat='Continuous')

## First set of binary variables 

y = LpVariable.dicts('BinVar1', product_list, cat='Binary')

## Second set of binary variables 

d = LpVariable.dicts('BinVar2', product_list, cat = 'Binary')

Setting Objective Function

In [23]:
model += sum(X[product] + G[product] + L[product] - S[product] for product in products)

Setting Constraints 

In [24]:
model += sum([X[product]*q[product] for product in products])<= B # budget constraints 

for product in product_list:
  
  model += X[product] <= I[product]

  model += S[product] <= D[product]
  model += S[product] <= X[product] + G[product] + L[product]
  
  model += X[product] >= (D[product] - L[product] - G[product])*y[product] + I[product] * (1-y[product])

  model += I[product] + G[product] + L[product] >= D[product] + 0.001 - M*(1-d[product])
  model += I[product] + G[product] + L[product] <=  D[product] + M*d[product]

  model += 1 - M*(1-d[product]) <= y[product]
  model += y[product] <= 1+ M*(1-d[product])

  model += 0 - M*d[product] <= y[product]
  model += y[product] <= 0 + M*d[product]


Solving the problem 

In [25]:
# Solve model

model.solve()


1

In [26]:
X_val = []
X_name = []

for v in model.variables():
    if "Supplied" in v.name:
      name = v.name.split('_')[1]
      X_name.append(name)
      X_val.append(v.varValue)

In [27]:
result = pd.DataFrame()
result['Name'] = X_name
result['Supplied'] = X_val

In [28]:
result['Garden']= result['Name'].map(G)
result['Previous Market Left-Overs']= result['Name'].map(L)

In [29]:
result['Total Inventory'] = result.sum(axis=1)
result['Demand']= result['Name'].map(D)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [30]:
Sales = {}

for v in model.variables():
    if "Sales" in v.name:
      name = v.name.split('_')[1]
      Sales[name] = v.varValue

result['Sales']= result['Name'].map(Sales)

In [31]:
result['Expected Left-Overs'] = result['Total Inventory'] - result['Sales']

In [32]:
result['empty 1'] = ""

result['empty 2'] = ""

result['empty 3'] = ""

result['empty 4'] = ""

result['excess inventory'] = result['Sales'] - result['Supplied']

result['Supplier Inventory'] = result['Name'].map(I)


In [33]:
result

,Name,Supplied,Garden,Previous Market Left-Overs,Total Inventory,Demand,Sales,Expected Left-Overs,empty 1,empty 2,empty 3,empty 4,excess inventory,Supplier Inventory
0,Apples,0.0,0.0,684.666667,684.666667,72,72.0,612.666667,,,,,72.0,97.0
1,Beans,0.0,0.0,193.500000,193.500000,0,0.0,193.500000,,,,,0.0,73.0
2,Beets,0.0,0.0,0.000000,0.000000,0,0.0,0.000000,,,,,0.0,39.0
3,Broccoli,0.0,0.0,0.000000,0.000000,0,0.0,0.000000,,,,,0.0,99.0
4,Cabbage,0.0,0.0,0.000000,0.000000,0,0.0,0.000000,,,,,0.0,26.0
5,Carrots,0.0,0.0,1627.800000,1627.800000,28,28.0,1599.800000,,,,,28.0,37.0
6,Celery,0.0,0.0,185.000000,185.000000,0,0.0,185.000000,,,,,0.0,91.0
7,Chickpeas,0.0,0.0,358.489000,358.489000,0,0.0,358.489000,,,,,0.0,79.0
8,Cucumber,69.0,0.0,0.000000,69.000000,111,69.0,0.000000,,,,,0.0,69.0
9,Eggs,0.0,0.0,1476.000000,1476.000000,5,5.0,1471.000000,,,,,5.0,29.0


In [34]:
# Upload Data to Google Sheets 
gc = gspread.authorize(creds)
sheet = client.open("DASHBOARD").worksheet('Forecasting DB')

cell_list = sheet.range('D5:N46')


to_googlesheets = result[['Name', 'empty 1', 'Supplier Inventory', 'Garden', 'Previous Market Left-Overs', 'empty 2', 'Demand' ,'Supplied', 'excess inventory', 'Sales', 'Expected Left-Overs']]
to_googlesheets = to_googlesheets.to_numpy().flatten()

for i in range(len(to_googlesheets)):
  cell_list[i].value = ''
  cell_list[i].value = to_googlesheets[i]

sheet.update_cells(cell_list)


{'spreadsheetId': '1tEU66cgvUdQ-jIt3p-RCx3qVV59xKlKWPzOlZdWYjoc',
 'updatedCells': 462,
 'updatedColumns': 11,
 'updatedRange': "'Forecasting DB'!D5:N46",
 'updatedRows': 42}